# Hangman

Retrouver tous les fichiers python sur le repo suivant => `https://github.com/MathiasGenibrel/Hangman`

## Étape 1 class Player

### Class Player

En premier temps, nous créons une class Player, pour implémenter les mêmes méthodes à tous les types de joueurs.
La class parent de tous les types de joueurs

In [47]:
class Player:
    """
        This class is abstract, **do not instantiate it**.\n
        It allows to implement methods common to all its children.
    """

    def __init__(self, username):
        self.username = username

    @staticmethod
    def get_secret_word(regex_pattern: str) -> str:
        """
        Getting secret word from Human or AI.

        :param str regex_pattern: Pattern to check if the user inputs are good or not
        :return: Secret word selected by the Player (Human | AI)
        """
        pass

    @staticmethod
    def guessing_letter(list_guessing_letters: list[str], regex_pattern: str, secret_word_guessing=None) -> str:
        """
        Getting a letter from Human or AI.

        :param list[str] list_guessing_letters: List of all letters already played by the player
        :param str regex_pattern: Pattern to check if the user inputs are good or not
        :param str secret_word_guessing: The hidden secret word, which must be discovered by the player
        :return: Letter selected by Player (Human | AI)
        """
        pass

### Class Human

Permet à un joueur de jouer au jeu

In [48]:
import re


class Human(Player):
    """
    This class allow a human player to play the Hangman game
    """

    def __init__(self, username):
        super().__init__(username)

    @staticmethod
    def get_secret_word(regex_pattern: str) -> str:
        secret_word = None

        while not secret_word:
            secret_word_input = str(input("Votre mot secret : "))

            if len(secret_word_input) < 4:
                print("Veuillez choisir un mot de plus de 4 caractères \n")
            elif len(secret_word_input) > 12:
                print("Veuillez choisir un mot de moins de 12 caractères \n")
            elif re.match(regex_pattern, secret_word_input) is None:
                print("Veuillez mettre que des lettres dans votre mot \n")
            else:
                secret_word = secret_word_input.lower()

        return secret_word

    @staticmethod
    def guessing_letter(list_guessing_letters: list[str], regex_pattern: str, secret_word_guessing=None) -> str:
        guessing_letter = None

        while guessing_letter is None:
            guessing_letter_input = str(input("\nChoisissez une lettre : ")).lower()

            # Validate data, control => length, pattern (regex) and if the letter has already played
            if 0 == len(guessing_letter_input) or len(guessing_letter_input) > 1:
                print("Veuillez ne choisir qu'une lettre ! [a, b, c, d, ...]")
            elif re.match(regex_pattern, guessing_letter_input) is None:
                print("Veuillez choisir une lettre !")
            elif guessing_letter_input in list_guessing_letters:
                print("Veuillez choisir une lettre que vous n'avez pas déjà sélectionnée !")
                print(f"Voici un rappel des lettre que vous avez joué : {list_guessing_letters}")
            else:
                guessing_letter = guessing_letter_input

        return guessing_letter


## Étape 2 : Une IA random

### Class AI

En premier temps, nous créons une class AI qui permettra d'avoir une base commune pour toutes les IA.
Sert de parent pour toutes les class AI qui suivent, et de définir une méthode pour donner un mot secret

In [49]:
class AI(Player):
    """
        Artificial Intelligence Player, This class init this AI with a default username ("Siri").\n
        This class is parents of different AI models present in this 'Project'.\n

        This class has a method so that the AI offers the player to guess the secret word.
    """

    def __init__(self, secret_word=None):
        # Give a name to AI
        super().__init__("Siri")

        # Get a list of different secret_word
        self.secret_word = secret_word

    def get_secret_word(self, regex_pattern: str) -> str:
        return self.secret_word


### Class AiRandom

L'intelligence artificielle la plus bête, joue les lettres de manière aléatoire.

In [50]:
from random import choice
from string import ascii_lowercase


class AiRandom(AI):
    """
        This Player is an AI, it plays randomly
    """

    def __init__(self, secret_word=None):
        super().__init__()

        self.list_letters = [letter for letter in ascii_lowercase]

    def guessing_letter(self, list_guessing_letters: list[str], regex_pattern: str, secret_word_guessing=None) -> str:
        guessing_letter = None

        while guessing_letter is None:
            guessing_letter_choice = choice(self.list_letters)

            if guessing_letter_choice not in list_guessing_letters:
                guessing_letter = guessing_letter_choice

        return guessing_letter


## Étape 3 : Une IA un peu plus logique
L'intelligence artificielle qui joue toutes les lettres qui ont le taux d'apparitions le plus élevé

### Class LetterUsageSoup

Permet de récupérer les lettres par taux d'apparition dans la langue française.

Pour le bon fonctionnement de cette class il faut d'abord récupérer les lettres avec le taux d'apparitions le plus élevé.
Nous effectuons cette tâche en utilisant une méthode de scrap, sur une page de [Wikipédia](https://fr.wikipedia.org/wiki/Fr%C3%A9quence_d%27apparition_des_lettres)

In [51]:
import string
import requests
from bs4 import BeautifulSoup


# List frequency letter get by scrapping wikipedia :
# (https://fr.wikipedia.org/wiki/Fr%C3%A9quence_d%27apparition_des_lettres)

class LetterUsageSoup:
    def __init__(self):
        # Use Wikipedia to get order of the most frequencies letter in French language.
        self.url = "https://fr.wikipedia.org/wiki/Fr%C3%A9quence_d%27apparition_des_lettres"

    def get_most_frequency_letters(self):
        soup = BeautifulSoup(self.__get_source_page(), "html.parser")

        # Get the first table, the first table corresponding to the frequency letter table
        frequency_letter_table = soup.find("table", class_="wikitable sortable")

        # Get all letter in the order (by default the table has been sorted to most frequently letter to the less)
        letter_by_most_frequency = [
            letter.text.strip("\n")
            for row in frequency_letter_table.find_all("tr")
            for letter in row.find_all("td") if letter.text.strip("\n") in string.ascii_lowercase
        ]

        return letter_by_most_frequency

    def __get_source_page(self):
        source_page = requests.get(self.url)

        return source_page.text


### Class AiDump

Et maintenant la class de l'intelligence artificielle, qui joue les lettres avec le taux d'apparition le plus élevé.

In [52]:
class AiDump(AI):
    def __init__(self):
        super().__init__()

        letter_usage = LetterUsageSoup()

        self.list_best_letter = letter_usage.get_most_frequency_letters()

    def guessing_letter(self, list_guessing_letters: list[str], regex_pattern: str, secret_word_guessing=None) -> str:
        # Return different letter each time.
        return self.list_best_letter[len(list_guessing_letters)]


## Étape 4 : Jouer intelligemment

L'intelligence artificielle dit intelligente, car en premiers temps, elle va jouer les lettres avec le taux d'apparition le plus élevé, jusqu'à trouver au moins une lettre.
Ensuite, à l'aide des lettres trouvé ainsi que de sa connaissance de la longueur du mot secret, elle va scrapper le web à la recherche d'une liste de mots qui pourraient correspondre.

Avec cette liste de mots, elle va calculer un taux de probabilité pour chacune des lettres et jouer la lettre avec le taux le plus élevé.

### Class WordFinder

Mais tout d'abord nous devons implémenter la class qui va nous permettre de scrapper le web à la recherche de notre liste de mots.

In [53]:
import requests
from bs4 import BeautifulSoup


class WordFinder:
    def __init__(self, word_check: int = 1000):
        # The data is provided from this website -> "https://scrabbledb.com/"
        # The param -> q is usage for pass the letter is in the secret word.
        # Use r param to change the size of the word table
        self.url = "https://scrabbledb.com/api.php"
        self.params = {
            "s": "fr_aspell",
            "l": "fr",
            "r": word_check,
            "q": str()
        }

        # Caching data
        self.previous_letters_found = []
        self.previous_word_list = []

    def get_list_word(self, letter_found: list[str], length_secret_word: int) -> list[str] or str:
        """
        Get a list of all words possibility with the current letter found and the length of the secret word.
        :param list[str] letter_found: List of letters found that are present in the secret word.
        :param int length_secret_word: Total length of the secret word.
        :return: Return a list with all words found, or return a string with an error message.
        """

        if self.previous_letters_found == letter_found:
            return self.previous_word_list

        # Update the query param, for search word, cannot be None and the max length is 12.
        self.params["q"] = self.__get_global_letters(letter_found, length_secret_word)

        # Try if we can request get the source code on the context url.
        try:
            # Update previous word list and previous letter found, for caching result for next request
            self.previous_word_list = self.__get_words_list(length_secret_word)
            self.previous_letters_found = letter_found

            return self.previous_word_list
        except ConnectionError as error:
            # Return a Message Error (only catch ConnectionError)
            return f"Connection error : {error}"

    def __get_source_code(self) -> str:
        """
        Get the source code of the context url
        :return: String contains the source code, on string format.
        """
        source_page = requests.get(self.url, params=self.params)

        return source_page.text

    def __get_words_list(self, length_secret_word: int) -> list[str] or ConnectionError:
        """
        Get words list from a web page.
        :param int length_secret_word: Use this param to remove all word not corresponding at our secret word.
        :return: A list of words that could match our secret word.
        """
        try:
            source_code = self.__get_source_code()
        except requests.exceptions.ConnectionError:
            raise ConnectionError("Une erreur avec le serveur distant est survenue, Veuillez ressayer plus tard.")

        soup = BeautifulSoup(source_code, "html.parser")

        # Get all words equal to the same length of secret word, in the web page
        words = [
            word.text.lower()
            for word in soup.find_all("a", {"id": "wordLink"})
            if len(word.text) == length_secret_word
        ]

        return words

    @staticmethod
    def __get_global_letters(letter_found: list[str], length_secret_word: int) -> str:
        """
        Format letter present in the secret word and the length of the secret word, in string ready for the request.
        \n
        Example: letter_found : ["a","b","c"], length_secret_word: 6 => return "abc***"
        :param list[str] letter_found: List of letters found that are present in the secret word.
        :param int length_secret_word: Total length of the secret word.
        :return: Formatted string (to match api format) => "abc***"
        """
        # Concat list
        global_letters_list = [] + letter_found

        for index in range(len(letter_found), length_secret_word):
            global_letters_list.insert(index, "*")

        return "".join(global_letters_list)


### Class AiSmart

Voir ce qui est dit au-dessus

In [54]:
from models.abstract.AI import AI
from models.LetterUsageSoup import LetterUsageSoup
from models.WordFinder import WordFinder


class AiSmart(AI):
    def __init__(self):
        super().__init__()

        letter_usage = LetterUsageSoup()

        self.list_best_letter = letter_usage.get_most_frequency_letters()
        self.words_finder = WordFinder()

    def guessing_letter(self, list_guessing_letters: list[str], regex_pattern: str, secret_word_guessing=None) -> str:
        letters_found = [letter for letter in secret_word_guessing if letter != "_"]

        if len(list_guessing_letters) == 0 or len(letters_found) == 0:
            guessing_letter = self.list_best_letter[len(list_guessing_letters)]
        else:
            words_list = self.words_finder.get_list_word(
                letter_found=letters_found,
                length_secret_word=len(secret_word_guessing)
            )
            clean_words_list = self.__remove_incorrect_words(words_list, secret_word_guessing)

            if len(clean_words_list) > 0:
                guessing_letter = self.__get_letter_probability(clean_words_list, list_guessing_letters)
            # If no words match with the letter position of our secret_word, user basic words_list
            else:
                guessing_letter = self.__get_letter_probability(words_list, list_guessing_letters)

        return guessing_letter

    def __get_letter_probability(self, words_list: list[str], list_guessing_letters: list[str]) -> str:
        """
        Get the probability of each letter that could potentially be in the secret word.\n

        If the words list contains no potential words, we use the list with high frequency letters.
        :param list[str] words_list: List of all the words that can potentially be the secret word
        :param list[str] list_guessing_letters: List of all letters already played by the player
        :return: The letter with the best probability.
        """
        all_letter = [
            letter
            for word in words_list
            for letter in word
            if letter not in list_guessing_letters
        ]

        count_letter = {letter: all_letter.count(letter) for letter in all_letter}

        # The dictionary that contains the letters with the highest probability rate.
        sorted_count_letter = self.__get_probability_dictionary(count_letter)

        # If the dictionary is empty, we return a list based on list with high frequency letters
        if len(sorted_count_letter) == 0:
            return [letter for letter in self.list_best_letter if letter not in list_guessing_letters][0]

        # Return list of letter by best probability
        return [letter for letter in sorted_count_letter][0]

    @staticmethod
    def __remove_incorrect_words(words_list, secret_word_guessing) -> list[str]:
        """
        Remove words not match with the position of letter in the secret word.
        :param words_list: List of all the words that can potentially be the secret word
        :param secret_word_guessing: Secret word, to know the position of the letters that have been guessed
        :return: List of words can be the current secret word, can be emtpy list if no words in the list match.
        """
        # Get position of each letter in the secret word.
        enumerate_secret_word = [(index, letter) for index, letter in enumerate(secret_word_guessing) if letter != "_"]

        list_correct_potential_word = []

        # Control each position of letter in word of the list word
        # If one letter is not in same the position of secret_word
        # we don't add it on the correct potential word.
        for word in words_list:
            potential_correct_word = True
            for index, letter in enumerate_secret_word:
                if letter != word[index]:
                    potential_correct_word = False
                    break
            if potential_correct_word is True:
                list_correct_potential_word.append(word)

        return list_correct_potential_word

    def __get_probability_dictionary(self, dictionary):
        """
        Get dictionary with the highest probability letter
        :param dictionary: dictionary generate with list of word (scrapping)
        :return: dictionary with the highest probability letter
        """
        # Sorted the dictionary in descending order
        sorted_by_value = self.__sort_dictionary_by_value(dictionary)

        if len(sorted_by_value) <= 1:
            return sorted_by_value

        # Retrieves the first two elements, in order to compare them
        first_value = list(sorted_by_value.values())[0]
        second_value = list(sorted_by_value.values())[1]

        if first_value == second_value:
            # Get dictionary to represent value of letter by frequency in current language.
            best_letter_value = {
                letter: len(self.list_best_letter) - value
                for value, letter in enumerate(self.list_best_letter)
            }

            for letter, value in sorted_by_value.items():
                if value == first_value and letter in self.list_best_letter:
                    # Update the value and sort the dictionary again
                    sorted_by_value.update({letter: value + best_letter_value[letter]})
                    sorted_by_value = self.__sort_dictionary_by_value(sorted_by_value)

        # Return the dictionary without equality.
        return sorted_by_value

    @staticmethod
    def __sort_dictionary_by_value(dictionary):
        return dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=True))


## Class Hangman

Permet de jouer au jeu

In [55]:
from unidecode import unidecode


class Hangman:
    """
    Create a new Hangman instance game, you need to pass as argument, 2 players.\n
    This class wait 1 Player to be Game Master (Host the game, and set the secret word), and
    1 other Player to be de Guesser (say letter by letter, to find the secret word).\n

    We initialize this class with an "error_guessing_allowed" argument with a default value to `8`,
    this argument allow the Guesser to have **n** mistake.
    """

    def __init__(self, game_master: Player, guesser: Player, error_guessing_allowed: int = 8):
        # Secret word Section
        self.secret_word = None
        self.secret_word_guessing = None

        # Guessing Section
        self.error_guessing_remaining = error_guessing_allowed
        self.list_guessing_letters = []

        # init Player
        self.game_master = game_master
        self.guesser = guesser

        # Other param
        self.regex_pattern = r"[A-Za-zÀ-ÿ]+$"

    def start_game(self):
        """
        This method is used to start the Hangman Game
        :return: None
        """

        # Init the game with the secret word
        secret_word = self.game_master.get_secret_word(regex_pattern=self.regex_pattern)

        self.secret_word = secret_word
        self.secret_word_guessing = "".join(["_" for _ in secret_word])

        # While the secret word or life
        while self.secret_word_guessing != self.secret_word and self.error_guessing_remaining > 0:
            self.__print_info_player(f"Le mot à deviner : {self.secret_word_guessing}")
            self.__print_info_player(f"Vous avez {self.error_guessing_remaining} vie(s)")

            guessing_letter = self.guesser.guessing_letter(regex_pattern=self.regex_pattern,
                                                           list_guessing_letters=self.list_guessing_letters,
                                                           secret_word_guessing=self.secret_word_guessing
                                                           )
            # Add letter to list of 'already played letter'
            self.list_guessing_letters.append(guessing_letter)

            # Verify if the letter is in the secret word
            validation_guessing = self.__validation_guessing(guessing_letter)

            if validation_guessing is not None:
                self.__update_guessing_word(validation_guessing)

        if self.secret_word_guessing == self.secret_word:
            self.__print_info_player(
                f"Felicitation vous avez gagné, le mot à deviner était bien \"{self.secret_word}\"")
            return True
        else:
            self.__print_info_player("Vous avez perdu #HANG-MAN")
            self.__print_info_player(f"Vous avez jouez ces lettres : {self.list_guessing_letters}")
            return False

    def __validation_guessing(self, guessing_letter) -> list[tuple[int, str]] or None:
        """
        Verify if the current **guessing_letter** is on the current **secret_word**.\n
        If is not in the current secret_word, the guesser (Player who guess the secret word)
        loose 1 try.
        :param str guessing_letter: Letter selected by the Guesser
        :return: List with index and letter to change in the `secret_word_guessing` or `None`
        """
        all_secret_letter = [
            (index, letter)
            for index, letter in enumerate(self.secret_word)
            if unidecode(letter) == guessing_letter
        ]

        if len(all_secret_letter) > 0:
            return all_secret_letter

        self.error_guessing_remaining -= 1

    def __update_guessing_word(self, letter_to_display: list[tuple[int, str]]):
        """
        Update the `secret_word_guessing` according to the `letter_to_display`
        :param list[tuple[int, str]] letter_to_display: Letter to display in the `secret_word_guessing`
        :return: None
        """
        list_letter = [letter for letter in self.secret_word_guessing]
        for index, letter in letter_to_display:
            list_letter.pop(index)
            list_letter.insert(index, letter)

        self.secret_word_guessing = "".join(list_letter)

    def __print_info_player(self, message):
        if isinstance(self.guesser, Human) or isinstance(self.game_master, Human):
            print(message)

## Avant de commencer l'étape 5 :

Il va falloir créer une class afin de créer une liste de 1000 mots.

Cette class va créer un fichier JSON qui va contenir tous les mots secrets qui devront être trouvé par les IA.

In [56]:
import re
import time
import json
import string
import requests

from math import floor
from random import choice, randint
from bs4 import BeautifulSoup


class WordListGenerator:
    def __init__(self):
        # Same regex at Hangman class
        self.regex_pattern = r"[A-Za-zÀ-ÿ]+$"

        # Set default attributs
        self.url = "https://www.le-dictionnaire.com/repertoire/"
        self.list_letter = [letter for letter in string.ascii_lowercase]
        self.max_page_path = {}

        # Update self.max_page_path, to it's initial value.
        self.__get_max_page_path()

        # Set value for the word list file
        self.filename = "WordList"
        self.file_format = "json"

        # Contains all select word by algorithm
        self.selected_words = []

    def generate_file(self):
        while len(self.selected_words) < 1000:
            self.__get_words()

        with open(f"{self.filename}.{self.file_format}", "w+") as file:
            json.dump(dict(words=self.selected_words), file)

            print("\nFile Created ! 🎉")
            print(f"The file was named : '{self.filename}.{self.file_format}'")

    def get_data(self):
        with open(f"{self.filename}.{self.file_format}", "r") as file:
            json_data = json.load(file)

        return json_data["words"]

    # Refactor this into 1 parent class (this method was used in 3 différent files)
    @staticmethod
    def __get_source_code(url, params=None):
        return requests.get(url, params=params).text

    def __get_words(self):
        current_letter = choice(self.list_letter)
        current_page = randint(1, self.max_page_path[current_letter])
        current_url = f"{self.url}{current_letter}{current_page:02d}"

        soup = BeautifulSoup(self.__get_source_code(url=current_url), "html.parser")
        words_list = [
            link.text
            for link in soup.find_all("a")
            if "/definition/" in link.get("href")
               and re.match(self.regex_pattern, link.text)
               and 4 <= len(link.text) <= 12
        ]

        self.selected_words += words_list

    def __get_max_page_path(self):
        print("Preparation of letter path dictionary ...")
        for letter in self.list_letter:
            time.sleep(0.25)
            # Build url, this current url correspond to this (for letter 'a'):
            # "https://www.le-dictionnaire.com/repertoire/a01"
            current_url = f"{self.url}{letter}01"

            soup = BeautifulSoup(self.__get_source_code(current_url), "html.parser")

            paths_current_letter = [
                link.get("href")
                for link in soup.find_all("a")
                if f"/repertoire/{letter}" in link.get("href")
            ]

            self.max_page_path[letter] = len(paths_current_letter)
            print(f"In progress ...{floor(len(self.max_page_path) / len(self.list_letter) * 100)}%")


## Étape 5 : Simulation de 1000 partie.

Dans le code ci-dessous, j'ai limité l'example à 10 mots, car pour 1000 mots, il faut plus d'une heure afin que tous les mots soient testés.

In [59]:
words_list_generator = WordListGenerator()
words_list_generator.generate_file()

artificial_intelligence_result = {
    "AiRandom": [],
    "AiDump": [],
    "AiSmart": [],
}

# For the example we limited at 10 secret word, because with 1000 words the test of all the AI took more than an hour
words_list = words_list_generator.get_data()[:10]

ai_random = AiRandom()
ai_dump = AiDump()
ai_smart = AiSmart()

for index in range(len(words_list)):
    game_master = AI(secret_word=words_list[index])

    print(f"\n{index + 1} / {len(words_list)}")

    print("\nAiRandom : ")
    print("Loading ...")
    artificial_intelligence_result["AiRandom"].append(Hangman(game_master, ai_random).start_game())
    print("Done ✅")

    print("\nAiDump : ")
    print("Loading ...")
    artificial_intelligence_result["AiDump"].append(Hangman(game_master, ai_dump).start_game())
    print("Done ✅")

    print("\nAiSmart : ")
    print("Loading ...")
    artificial_intelligence_result["AiSmart"].append(Hangman(game_master, ai_smart).start_game())
    print("Done ✅")


Preparation of letter path dictionary ...
In progress ...3%
In progress ...7%
In progress ...11%
In progress ...15%
In progress ...19%
In progress ...23%
In progress ...26%
In progress ...30%
In progress ...34%
In progress ...38%
In progress ...42%
In progress ...46%
In progress ...50%
In progress ...53%
In progress ...57%
In progress ...61%
In progress ...65%
In progress ...69%
In progress ...73%
In progress ...76%
In progress ...80%
In progress ...84%
In progress ...88%
In progress ...92%
In progress ...96%
In progress ...100%

File Created ! 🎉
The file was named : 'WordList.json'

1 / 10

AiRandom : 
Loading ...
Done ✅

AiDump : 
Loading ...
Done ✅

AiSmart : 
Loading ...
Done ✅

2 / 10

AiRandom : 
Loading ...
Done ✅

AiDump : 
Loading ...
Done ✅

AiSmart : 
Loading ...
Done ✅

3 / 10

AiRandom : 
Loading ...
Done ✅

AiDump : 
Loading ...
Done ✅

AiSmart : 
Loading ...
Done ✅

4 / 10

AiRandom : 
Loading ...
Done ✅

AiDump : 
Loading ...
Done ✅

AiSmart : 
Loading ...
Done ✅

5 / 1

## Étape 6 : Les stats

Les stats sont calculée en fonction des résultats si le mot à été trouvé ou non.

Résultat que j'ai pu optenir avec 1000 mots, (les mots sont disponibles dans le fichier json "WordListStats.json"):
```json
{
    "AiRandom": "0.00%",
    "AiDump": "27.00%",
    "AiSmart": "68.00%"
}
```

In [61]:
ai_statistics = {
    ai: "{:.2f}%".format(round(result.count(1) / len(result) * 100))
    for ai, result in artificial_intelligence_result.items()
}

print(ai_statistics)

{'AiRandom': [False, False, False, False, False, False, False, False, False, False], 'AiDump': [False, False, False, False, False, False, False, False, False, False], 'AiSmart': [True, True, True, True, True, True, True, False, False, True]}
{'AiRandom': '0.00%', 'AiDump': '0.00%', 'AiSmart': '80.00%'}
